In [ ]:
# import libraries
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D, Activation, Multiply, Conv2D, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from numpy.random import seed
from helper_functions import list_labels, load_data, insert_layer_nonseq, inference_time
import seaborn as sn
# import mobilenet

tf.config.experimental.enable_op_determinism()

Attention Modules

In [ ]:
# the three attention mechanisms cited from https://github.com/nikhilroxtomar/Attention-Mechanism-Implementation.git
# channel module
def channel_attention_module(x, ratio, bias):
    b, _, _, channel = x.shape
    # MLP shared layer
    l1 = Dense(channel//ratio, activation="relu", use_bias=bias)
    l2 = Dense(channel, use_bias=bias)
    
    # Global Average pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)
    
    # Global Max pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)
    
    # Adding both and applying sigmoid
    features = x1 + x2
    features = Activation("sigmoid")(features)
    features = Multiply()([x, features])
    
    return features

# spatial attention module
def spatial_attention_module(x, kernel_size=7, bias=True):
    # Average pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)
    
    # Max pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    
    # Concatenate
    features = Concatenate()([x1, x2])
    
    # Conv layer
    features = Conv2D(1, kernel_size=kernel_size, padding="same", activation="sigmoid", use_bias=bias)(features)
    features = Multiply()([x, features])
    
    return features
    
# CBAM
def CBAM(x, ratio=16, bias=True):
    x = channel_attention_module(x, ratio=ratio, bias=bias)
    x = spatial_attention_module(x)
    return x

def CBAM_factory():
    return CBAM
    

Loading data

In [ ]:
#loading and shuffling dataset
#list of all labels
class_names = list_labels("./CamSDD/Labels.txt")
class_name_labels = {class_name:i for i, class_name in enumerate(class_names)}

(train_images, train_labels), (test_images, test_labels), (validation_images, validation_labels)= load_data("./CamSDD", class_name_labels)
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
validation_images, validation_labels = shuffle(validation_images, validation_labels, random_state=25)

Creating and training model

In [ ]:
#seed
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)

#creating model using mobilenet function
base_model = MobileNet(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
'''
uncomment below to use mobilenet with custom activations
'''
# base_model = mobilenet.MobileNet(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

'''
Uncomment the line below to insert attention modules
The line below only adds a single attention
The second parameter has to be modified to add more 
'''
# base_model = insert_layer_nonseq(base_model, '.*pw_(13)_relu.*', CBAM_factory)
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='sigmoid')(x)
x=Dropout(0.7)(x)
output = Dense(30, activation="softmax")(x)
model=Model(inputs=base_model.input,outputs=output)

'''
Comment the two line below to not freeze the model
'''
for layer in base_model.layers:
    layer.trainable = False
    
# training the model with an early stopping monitor
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy", SparseTopKCategoricalAccuracy(k=3)])
history = model.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor], shuffle=False)

Evaluating trained model

In [ ]:
'''
Uncomment the line below to load a saved model
'''
model = keras.models.load_model('CBAM_combined_relu')

score = model.evaluate(test_images, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

average_inference_time = inference_time(model, test_images, test_labels)
print('Inference time:', average_inference_time)

In [ ]:
# confusion matrix
test_pred = model.predict(test_images)
test_pred = np.argmax (test_pred, axis = 1)
result = confusion_matrix(test_labels, test_pred)
print(result)

In [ ]:
# using seaborn to visualise confusion matrix
plt.figure(figsize = (10,6))
ids = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
results = pd.DataFrame(result, index = [i for i in ids],
                  columns = [i for i in ids])
ax = sn.heatmap(results, annot=True, cmap=sn.cubehelix_palette(as_cmap=True))